# Attendance_System.ipynb

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models

# Face Recognition Model Training

In [ ]:
def preprocess_images(image_folder):
    images=[]
    labels=[]
    for label,person_name in enumerate(os.listdir(image_folder)):
        person_path=os.path.join(image_folder,person_name)
        for img_file in os.listdir(person_path):
            img_path=os.path.join(person_path,img_file)
            img=cv2.imread(img_path)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img=cv2.resize(img,(128,128))  # Resize images
            images.append(img)
            labels.append(label)
    images=np.array(images).reshape(-1,128,128,1)  # Add channel for CNN
    labels=np.array(labels)
    return images,labels
X_faces,y_faces=preprocess_images('data/students/')
X_train,X_val,y_train,y_val=train_test_split(X_faces,y_faces,test_size=0.2)

# Build a CNN model for face recognition

In [ ]:
face_model=models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(len(np.unique(y_faces)),activation='softmax')
])
face_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
face_model.fit(X_train,y_train,epochs=10,validation_data=(X_val,y_val))

# Save the trained face recognition model

In [ ]:
face_model.save('models/face_recognition_model.h5')

# Emotion Detection Model Training

In [ ]:
def preprocess_emotions(image_folder):
    images=[]
    labels=[]
    for label,emotion_name in enumerate(os.listdir(image_folder)):
        emotion_path=os.path.join(image_folder,emotion_name)
        for img_file in os.listdir(emotion_path):
            img_path=os.path.join(emotion_path,img_file)
            img=cv2.imread(img_path)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img=cv2.resize(img,(48,48))  # Resize images for emotion detection
            images.append(img)
            labels.append(label)
    images=np.array(images).reshape(-1,48,48,1)
    labels=np.array(labels)
    return images,labels
X_emotions,y_emotions=preprocess_emotions('data/emotions/')
X_train_emotions,X_val_emotions,y_train_emotions,y_val_emotions=train_test_split(X_emotions,y_emotions,test_size=0.2)

# Build a CNN model for emotion detection

In [ ]:
emotion_model=models.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(48,48,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(7,activation='softmax')  #any 7 emotions like happy, sad, etc.
])
emotion_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
emotion_model.fit(X_train_emotions,y_train_emotions,epochs=10,validation_data=(X_val_emotions,y_val_emotions))

# Save the trained emotion detection model

In [ ]:
emotion_model.save('models/emotion_detection_model.h5')